In [2]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import time
import random

import numpy as np
import pandas as pd
import scipy as sp

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm

import matplotlib.pyplot as plt

from sktime.datasets import load_UCR_UEA_dataset

/home/schlegel/.conda/captum/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import exp_attributions as exp_att
import exp_perturbation_analysis as exp_pa
import exp_perturbation_card as exp_card

In [5]:
random_seed = 13

torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
dataset = 'FordA'
dataset_name = dataset.lower()

cur_time = time.strftime('%Y-%m-%d_%H-%M-%S')
base_dir = f'./results/{dataset_name}--{cur_time}'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
X_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train', return_type='numpyflat')
X_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test', return_type='numpyflat')

print(f'Length training data: {len(X_train)} labels: {len(y_train)} test data: {len(X_test)} labels: {len(y_test)}')

In [8]:
encoder = OneHotEncoder(categories='auto', sparse=False)

y_train_ohe = encoder.fit_transform(np.expand_dims(y_train, axis=-1))
y_test_ohe = encoder.transform(np.expand_dims(y_test, axis=-1))

y_train_norm = y_train_ohe.argmax(axis=-1)
y_test_norm = y_test_ohe.argmax(axis=-1)

/home/schlegel/.conda/captum/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
class FordADataset(Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        inputs = self.X[idx]
        label = self.y[idx]
        
        return inputs, label

In [10]:
dataset_train = FordADataset(X_train, y_train_ohe)
dataset_test = FordADataset(X_test, y_test_ohe)

In [11]:
dataloader_train = DataLoader(dataset_train, batch_size=120, shuffle=True)
dataloader_train_not_shuffled = DataLoader(dataset_train, batch_size=120, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=120, shuffle=False)

In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 10, kernel_size=3, stride=1),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(10, 50, kernel_size=3, stride=1),
            nn.MaxPool1d(3),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(50, 100, kernel_size=3, stride=1),
            nn.MaxPool1d(3),
            nn.ReLU(inplace=True)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(100 * 54, 100),
            nn.Dropout(0.5),
            nn.ReLU(inplace=True)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(100, 2),
            nn.Softmax(-1)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [13]:
def trainer(model, dataloader_train, criterion):
    running_loss = 0

    model.train()

    for idx, (inputs, labels) in enumerate(dataloader_train):
        inputs = inputs.reshape(inputs.shape[0], 1, -1)
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        optimizer.zero_grad()
        preds = model(inputs)
        loss = criterion(preds, labels.argmax(dim=-1))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    train_loss = running_loss / len(dataloader_train)
    
    return train_loss


def validator(model, dataloader_test, criterion):
    running_loss = 0

    model.eval()

    for idx, (inputs, labels) in enumerate(dataloader_test):
        inputs = inputs.reshape(inputs.shape[0], 1, -1)
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        preds = model(inputs)
        loss = criterion(preds, labels.argmax(dim=-1))
        
        running_loss += loss.item()

    train_loss = running_loss / len(dataloader_train)
    
    return train_loss

In [14]:
model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss = nn.CrossEntropyLoss()

In [15]:
epochs = 500

for epoch in range(epochs):
    train_loss = trainer(model, dataloader_train, loss)
    if epoch % 10 == 0:
        print('Val', validator(model, dataloader_test, loss))

Val 0.24538052466607863
Val 0.2271355063684525
Val 0.19523285473546675
Val 0.1737615898732216
Val 0.16192963334821886
Val 0.15819191932678223
Val 0.15484415235057955
Val 0.15367361518644518
Val 0.150385546107446
Val 0.15013589974372618
Val 0.14899024559605506
Val 0.14794734120368958
Val 0.1470267628469775
Val 0.14894994131980405
Val 0.14720758411192125
Val 0.14759679763547837
Val 0.14933574968768704
Val 0.14568548625515354
Val 0.14541851897393504
Val 0.14596126637151163
Val 0.14974524032685063
Val 0.14596286319917248
Val 0.1468512983091416
Val 0.14665600753599597
Val 0.14690742954131095
Val 0.14606425646812685
Val 0.14628386593634082
Val 0.1456132761893734
Val 0.1460236003321986
Val 0.14587083843446547
Val 0.146202600771381
Val 0.14619139317543275
Val 0.14556700760318386
Val 0.14618127961312571
Val 0.1455558269254623
Val 0.14657215918264083
Val 0.14776922618189164
Val 0.1462692810643104
Val 0.14749211938150467
Val 0.1466430233370873
Val 0.15436521845479165
Val 0.14800879166972253
Val 0

In [16]:
model.eval()

preds = []
labels = []
for x in dataloader_train_not_shuffled:
    input_, label_ = x
    input_ = input_.reshape(input_.shape[0], 1, -1)
    input_ = input_.float().to(device)
    label_ = label_.float().to(device)

    pred_ = model(input_)
    preds.extend(pred_)
    labels.extend(label_)

preds = torch.stack(preds)
labels = torch.stack(labels)
print('Prediction Accuracy Train', np.round((preds.argmax(dim=-1) == labels.argmax(dim=-1)).int().sum().float().item() / len(preds), 4))

y_train_pred = preds.cpu().detach().numpy()

Prediction Accuracy Train 0.9936


In [17]:
model.eval()

preds = []
labels = []
for x in dataloader_test:
    input_, label_ = x
    input_ = input_.reshape(input_.shape[0], 1, -1)
    input_ = input_.float().to(device)
    label_ = label_.float().to(device)

    pred_ = model(input_)
    preds.extend(pred_)
    labels.extend(label_)

preds = torch.stack(preds)
labels = torch.stack(labels)
print('Prediction Accuracy Test', np.round((preds.argmax(dim=-1) == labels.argmax(dim=-1)).int().sum().float().item() / len(preds), 4))

y_test_pred = preds.cpu().detach().numpy()

Prediction Accuracy Test 0.8894


# Generate attributions for the training data

In [19]:
model.eval()

SimpleCNN(
  (conv1): Sequential(
    (0): Conv1d(1, 10, kernel_size=(3,), stride=(1,))
    (1): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv1d(10, 50, kernel_size=(3,), stride=(1,))
    (1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(3,), stride=(1,))
    (1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
  )
  (fc1): Sequential(
    (0): Linear(in_features=5400, out_features=100, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (fc2): Sequential(
    (0): Linear(in_features=100, out_features=2, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [20]:
sample, label = dataset_train[0]
shape = sample.reshape(1, -1).shape
baselines = torch.from_numpy(np.array([dataset_train[torch.randint(len(dataset_train), (1,))][0] for _ in range(10)])).reshape(-1, *shape).float().to(device)

In [21]:
attributions_train = {}
predictions_train = {}

attr_batch, preds_batch = exp_att.generate_attributions_batch(shape, model, device, dataloader_train_not_shuffled, baselines)
attributions_train.update(attr_batch)
predictions_train.update(preds_batch)

del attr_batch
del preds_batch

attr_single, preds_single = exp_att.generate_attributions_single(shape, model, device, dataloader_train_not_shuffled, baselines)
attributions_train.update(attr_single)
predictions_train.update(preds_single)

del attr_single
del preds_single

Start with DeepLiftShap             :   0%|          | 0/31 [00:00<?, ?it/s]/home/schlegel/.conda/captum/lib/python3.10/site-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
Start with Saliency                 :   0%|          | 0/31 [00:00<?, ?it/s]/home/schlegel/.conda/captum/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
Start with KernelShap               : 100%|██████████| 31/31 [06:20<00:00, 12.29s/it]


In [22]:
overall_results = exp_pa.perturbation_analysis(attributions_train, X_train, y_train, model, device, base_dir)

Start with delete_to_mean 1         : 100%|██████████| 2/2 [07:05<00:00, 212.70s/it]


In [ ]:
for cur_attr_name in attributions_train:
    cur_attr_name = cur_attr_name.lower().replace(' ', '_')
    cur_dir = f'{base_dir}/method-{cur_attr_name}'
    for deletion_value in exp_pa.values:
        deletion_value_fnc, deletion_length = deletion_value
        name = deletion_value_fnc.__name__ + ' ' + str(deletion_length)
        attribution = cur_attr_name
        
        results = overall_results[cur_attr_name][name]
        exp_card.create_perturbation_analysis_card(dataset, attribution, name, X_train, results, cur_dir)

In [79]:
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.int64):
            return str(obj)
        if isinstance(obj, np.float32):
            return str(obj)
        return JSONEncoder.default(self, obj)

overall_results_json = json.dumps(overall_results, cls=NumpyArrayEncoder)

with open(f'{base_dir}/results.json', 'w') as f:
    f.write(overall_results_json)

In [ ]:
import shutil

shutil.make_archive(f'./results/time-{cur_time}', 'zip', base_dir)